In [44]:
import asyncio
import json
from typing import List
from datetime import datetime, timedelta

from grpclib.client import Channel
from cortexadmin_pb import CortexAdminStub, SeriesRequest, QueryRangeRequest, QueryRequest, MatcherRequest
from betterproto.lib.google.protobuf import Empty

default_query_interval = "1m"

async def get_all_users(service: CortexAdminStub) -> List[str]:
  response = await service.all_user_stats(Empty())
  return [r.user_id for r in response.items]

async def list_all_metric(service: CortexAdminStub, cluster_id: str) -> List[str]:
  response = await service.extract_raw_series(MatcherRequest(tenant=cluster_id, match_expr=".+"))
  res = (json.loads(response.data.decode())["data"])
  s = set()
  for r in res["result"]:
    s.add(r["metric"]["__name__"])
  return list(s)

async def metric_query(service: CortexAdminStub, cluster_id: str, metric_name: str, namespace="opni"):
  query = f'sum(rate({metric_name}{{namespace="{namespace}"}}[{default_query_interval}])) by (pod)'
  response = await service.query(QueryRequest(tenants=[cluster_id], query=query))
  response = json.loads(response.data.decode())["data"]
  return response

async def metric_queryrange(service: CortexAdminStub, cluster_id: str, metric_name: str, namespace="opni", end_time : datetime = None, time_delta : timedelta= timedelta(minutes=60), step_minute : int = 1):
  query = f'sum(rate({metric_name}{{namespace="{namespace}"}}[{default_query_interval}])) by (pod)'
  if end_time is None:
    end_time = datetime.now()
  start_time = end_time - time_delta
  response = await service.query_range(QueryRangeRequest(tenants=[cluster_id], query=query, start=start_time, end=end_time, step=timedelta(minutes=step_minute)))
  response = json.loads(response.data.decode())["data"]
  return response

In [39]:
channel = Channel(host="localhost", port=11090) # url of opni-internal. can port-forward to localhost:11090
service = CortexAdminStub(channel)

In [40]:
user_id = (await get_all_users(service))[0]
user_id

'6165ba24-e496-491d-86f2-6eb083f24f0b'

In [36]:
metrics = await list_all_metric(service, user_id)

In [45]:
m_name = "container_cpu_usage_seconds_total"
q1 = await metric_query(service, user_id, m_name)
q1

sum(rate(container_cpu_usage_seconds_total{namespace="opni"}[1m])) by (pod)


{'resultType': 'vector',
 'result': [{'metric': {'pod': 'opni-prometheus-node-exporter-6zq4r'},
   'value': [1678496521.048, '0.0027759730305161965']},
  {'metric': {'pod': 'etcd-0'},
   'value': [1678496521.048, '0.06163341022026947']},
  {'metric': {'pod': 'opni-kube-prometheus-stack-operator-b6bd94cb-vn2f7'},
   'value': [1678496521.048, '0.00043594295307532654']},
  {'metric': {'pod': 'opni-prometheus-node-exporter-ljdr6'},
   'value': [1678496521.048, '0.0030721393809542497']},
  {'metric': {'pod': 'opni-manager-5f9b874fb8-75xmf'},
   'value': [1678496521.048, '0.006987418350081357']},
  {'metric': {'pod': 'opni-agent-0'},
   'value': [1678496521.048, '0.0544418983978075']},
  {'metric': {'pod': 'opni-gateway-799dfbf9cf-txnfh'},
   'value': [1678496521.048, '0.08390518893702333']},
  {'metric': {'pod': 'cortex-all-0'},
   'value': [1678496521.048, '0.597503780990446']},
  {'metric': {'pod': 'opni-prometheus-node-exporter-wx5gx'},
   'value': [1678496521.048, '0.0014154184087185738